## Настройка параметров и осмотр таблиц
<a id='rev'></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
q_books = ''' SELECT *
            FROM books
            LIMIT 5;
        '''    

In [5]:
data = pd.io.sql.read_sql(q_books, con = engine) #подключение к таблице с книгами
data #вывод первых 5 строк

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
q_authors = ''' SELECT *
            FROM authors
            LIMIT 5;
        '''

In [7]:
data = pd.io.sql.read_sql(q_authors, con = engine) #подключение к таблице с авторами
data #вывод первых 5 строк

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
q_publishers = ''' SELECT *
            FROM publishers
            LIMIT 5;
        '''

In [9]:
data = pd.io.sql.read_sql(q_publishers, con = engine) #подключение к таблице с издателями
data #вывод первых 5 строк

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
q_ratings = ''' SELECT *
            FROM ratings
            LIMIT 5;
        '''

In [11]:
data = pd.io.sql.read_sql(q_ratings, con = engine) #подключение к таблице с оценками пользователей
data #вывод первых 5 строк

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
q_reviews = ''' SELECT *
            FROM reviews
            LIMIT 5;
        '''

In [13]:
data = pd.io.sql.read_sql(q_reviews, con = engine) #подключение к таблице с оьзорами 
data #вывод первых 5 строк

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Подсчет, количества книг, которые вышли после 1 января 2000 года
<a id='1'></a>

In [14]:
query = ''' SELECT COUNT(DISTINCT book_id)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''

In [15]:
data = pd.io.sql.read_sql(query, con = engine)
data

,count
0,819


**Вывод:** Всего было выпущено 819 книг после 1 января 2000 года.

## Подсчет количества обзоров и среднюю оценку каждой книги
<a id='2'></a>

In [16]:
query = ''' SELECT b.title, c_rw.rw, avg_rt.rt
            FROM books AS b LEFT JOIN 
                        (SELECT book_id, AVG(rating) AS rt
                        FROM ratings
                        GROUP BY book_id) AS avg_rt ON b.book_id = avg_rt.book_id
                        LEFT JOIN
                        (SELECT book_id, COUNT(review_id) AS rw
                        FROM reviews
                        GROUP BY book_id) AS c_rw ON b.book_id = c_rw.book_id
        '''

In [17]:
data = pd.io.sql.read_sql(query, con = engine)
data

,title,rw,rt
0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,Galápagos,2.0,4.500000
2,A Tree Grows in Brooklyn,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,The Prophet,4.0,4.285714
...,...,...,...
995,Alice in Wonderland,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,Christine,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


**Вывод:** Для каждой из книг был посчитан средний рейтинг и общее количество обзоров на книгу.

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц
<a id='3'></a>

In [18]:
query = ''' SELECT pb.publisher, COUNT(b.book_id)
            FROM books as b
            INNER JOIN publishers AS pb ON b.publisher_id = pb.publisher_id
            WHERE b.num_pages > 50
            GROUP BY pb.publisher_id
            ORDER BY count DESC
            LIMIT 1;
        '''

In [19]:
data = pd.io.sql.read_sql(query, con = engine)
data

,publisher,count
0,Penguin Books,42


**Вывод:** Издательством, которое выпустило наибольшее количество книг толще 50 страниц, является Penguin Books.

## Поиск автора с самой высокой средней оценкой книг
<a id='4'></a>

In [20]:
query1 = ''' SELECT au, AVG(r)
            FROM (SELECT a.author AS au, AVG(rating) AS r
                    FROM books AS b
                    INNER JOIN ratings AS rt ON b.book_id = rt.book_id
                    INNER JOIN authors AS a ON b.author_id = a.author_id
                    GROUP BY b.book_id, a.author
                    HAVING COUNT(rt.rating) > 50) as c
            GROUP BY au
            LIMIT 1;
        '''

In [21]:
data = pd.io.sql.read_sql(query1, con = engine)
data

,au,avg
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод:** Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré, с рейтингом - 4.283844.

## Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок
<a id='5'></a>

In [22]:
query = ''' SELECT round(AVG(cn),1)
            FROM (SELECT rv.username, COUNT(review_id) AS cn
                    FROM (SELECT username
                        FROM books as b
                        INNER JOIN ratings AS rt ON b.book_id = rt.book_id
                        GROUP BY username
                        HAVING COUNT(rating_id) > 50) AS c
                    INNER JOIN reviews AS rv ON rv.username = c.username
                    GROUP BY rv.username) AS avg
                
        '''

In [23]:
data = pd.io.sql.read_sql(query, con = engine)
data

,round
0,24.3


**Вывод:** Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3

## Общий вывод
<a id='concl'></a>

**Вывод:** В ходе анализа базы данных сервиса для чтения книг обнаружилось, что после 1 января 2000 года было издано 819 книг. Для каждой из книг в базе данных было посчитано количество обзоров на нее и средний рейтинг. Был выявлен издатель с самым большим количеством изданных книг, которые толще 50 страниц - Penguin Books. Также, были проанализированы авторы книг с 50 и более оценками, была посчитана средняя оценка их книг и выявилось, что самый высокий средний рейтинг у книг J.K. Rowling/Mary GrandPré с рейтингом 4.283844. В заклчение, было выявлены пользователи, которые поставили книгам более 50 оценок. Затем, было подсчитано среднее количество обзоров этих пользователей, которое составило 24.3 обзора в среднем.